# The Battle of Neighbourhoods - Capstone Prject

## Opening a café chain in a new city

## Preparations

First we install all needed libraries.

In [6]:
!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge wikipedia -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

We define some functions for repeated useage

In [7]:
def get_coords_from_osm(location):

    # create url for OSM search
    url = 'http://nominatim.openstreetmap.org/search.php?{}{}{}'.format('q=', location, '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # get coordinates
    lst = [response.get(coord) for coord in ['lat','lon']]
    return [float(i) for i in lst]

In [8]:
def get_surrounding_from_fs(df_list_location, categoryId, radius=200):
    
    LIMIT = 100 # limit of number of venues returned by Foursquare API

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        df_list_location[1]['Lat'],
        df_list_location[1]['Long'],
        categoryId,
        radius,
        LIMIT)
    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    
    nearby_venues = json_normalize(venues) # flatten JSON
    
    if nearby_venues.empty == True:
        return nearby_venues
    
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
  
    return nearby_venues

In [9]:
def get_category_type(row):
    try:
        ategories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We define our hidden credentials for Foursquare

In [26]:
# The code was removed by Watson Studio for sharing.

We import needed libraries

In [11]:
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import requests
import folium
import math

# Methodology - Data analysis

### Base data acquisition and refinement

First we determine the coordinates of the city center of Cologne

In [12]:
# Get coordinates from Cologne city center
k_lat, k_long = get_coords_from_osm('Köln')

print('The city center can by found at a latitude of {} and longitude of {} - data acquired from OSM'.format(k_lat, k_long))

The city center can by found at a latitude of 50.938361 and longitude of 6.959974 - data acquired from OSM


Second, we gather location data of KVB tramway stations and have a look at it

In [13]:
#CSV
#https://online-service.kvb-koeln.de/geoserver/OPENDATA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ODENDATA:haltestellen&outputFormat=csv

df_stations = pd.read_csv('https://online-service.kvb-koeln.de/geoserver/OPENDATA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ODENDATA:haltestellen&outputFormat=csv')
df_stations.head()

,FID,ASS,Name,Kurzname,Haltestellenbereich,Koordinate,Betriebsbereich,Linien
0,haltestellen.fid-5764443a_16b5a8f4c71_47af,111,Heumarkt,HMG,1,POINT (6.9604570988 50.9357563921),STRAB,1 7 9
1,haltestellen.fid-5764443a_16b5a8f4c71_47b0,112,Heumarkt,HMG,1,POINT (6.9595737676 50.9357045307),STRAB,1 7 9
2,haltestellen.fid-5764443a_16b5a8f4c71_47b1,113,Heumarkt,HMG,1,POINT (6.9574528147 50.9351045185),STRAB,5
3,haltestellen.fid-5764443a_16b5a8f4c71_47b2,114,Heumarkt,HMG,1,POINT (6.9579496548 50.9355407748),STRAB,5
4,haltestellen.fid-5764443a_16b5a8f4c71_47b3,121,Heumarkt,HMG,1,POINT (6.9603999897 50.9355099544),BUS,106 132 133


In [14]:
df_stations.shape

(1991, 8)

We cearly need to refine the data to be able to work with ir properly

Now we refine the data.
- Remove unneeded columns (information we don't need for our endeavour)
- Remove unneeded rows (stations for bus stops)
- Remove duplicat stops (occure when more than one stop point is at the same location)
- Split the coordinates to single latitute and longitude values
- Evaluate how much lines serve this station
- Calculate the distance of the station from the city center

In [15]:
# Remove dublicat stations
df_stations.drop_duplicates(subset='Name', inplace=True)
# Remove unneded columns
df_stations.drop(['FID', 'ASS', 'Kurzname', 'Haltestellenbereich'], axis=1, inplace=True)
# Remove bus stations
df_stations = df_stations[df_stations.Betriebsbereich != 'BUS']

# Refine coordinate information
ls_lat = []
ls_long = []
ls_lines = []
for row in df_stations.iterrows():
    a, b = row[1]['Koordinate'].replace('POINT (','').replace(')','').split()
    ls_long.append(a)
    ls_lat.append(b)
    c = len(row[1]['Linien'].split())
    ls_lines.append(c)

# Remove other unneded columns
df_stations.drop(['Koordinate', 'Betriebsbereich', 'Linien'], axis=1, inplace=True)

# Add coordinates and lines serving the stations to final dataframe
df_stations['Long'] = ls_long
df_stations['Lat'] = ls_lat
df_stations['Lines count'] = ls_lines

df_stations.reset_index(drop=True, inplace=True)

# Claculate distance from city center
d = []
for row in df_stations.iterrows():
    dx = 71.5 * (k_long - float(row[1]['Long']))
    dy = 111.3 * (k_lat - float(row[1]['Lat']))
    d.append(math.sqrt(dx**2 + dy**2))

df_stations['Distance'] = d

Our station dataset has now the dimensionality of

In [16]:
print(df_stations.shape)

(225, 5)


and looks like

In [17]:
df_stations.head()

,Name,Long,Lat,Lines count,Distance
0,Heumarkt,6.9604570988,50.9357563921,3,0.291943
1,Neumarkt,6.9481136124,50.935762763,3,0.895969
2,Poststr.,6.9500805214,50.9316916582,4,1.025377
3,Mauritiuskirche,6.9450683908,50.9346873217,1,1.141494
4,Rathaus,6.9595586023,50.9379333709,1,0.056102


### Visual inspection of the data

Now let's mark the stations found in the data set on a map to do a visual inspection. We want to find out if the data looks correct and complete and if it is correctly refined.
We should expect:
- many stations in the city itself
- fewer ones outside the city
- there should be some kind of line pattern discernable

In [18]:
# create map of Cologne
k_map = folium.Map(location=[k_lat, k_long], zoom_start=11)

# add markers to indicate stations
for lat, long, name in zip(df_stations['Lat'], df_stations['Long'], df_stations['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(long)],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1.0,
        parse_html=False).add_to(k_map)  

k_map

Looks good.<br>
We have a dense net of stations in the city itself and then radial spreading arms of stations where the single lines go into the suburbs.

Now that we have the main data available and in a format we can easily work with, we try to achiv our goal and find some locations for our business.

### Detailed data analysis on potential customers and competitors

The business plan forsees that the company will open several new cafés near relevant tram stations. The competition should not be to high, meaning that there should be as less as possible other bakeries or cafés in the vicinity of the station. As a plus for the location are seen schools and university sites near the station, since it is expected, that there will be a high count on possible customers in addition to the occasional customers.<br>
We assume that a customer leaving a tram will look for about 100m around a station for a bakery or café for a snak and/or a coffee to buy. Further we assume that pupils and students will exit a station wich is nur further than 300m away from their school/university building.<br>
<br>
A short exemplary evaluation of different search querries for Foursquare has shown, that the best most relevent reulst for competition will be deliverd by the API via a venues search for:
- bakery : 4bf58dd8d48988d16a941735
- café : 4bf58dd8d48988d16d941735

More venue categories will lead to less relevant results.<br>
<br>
The high potential customer site will be identifyed by the venue categories:
- university : 4d4b7105d754a06372d81259
- school : 4bf58dd8d48988d13b941735

<br>
<br>

We search for potentail high customer sites and competitors and display our findings on the map as follows: 
- red marker: competitor
- green marker: customer site


In [19]:
# school, university
catId_customer = '4bf58dd8d48988d13b941735,4d4b7105d754a06372d81259'

# cafe, bakery
catId_competitor = '4bf58dd8d48988d16d941735,4bf58dd8d48988d16a941735'

In [20]:
#Debug
df3 = df_stations.copy()

In [21]:
#Debug
df_stations = df3.copy()
print(df_stations.shape)
#df_stations = df_stations[df_stations['Distance'] < 2.8]
print(df_stations.shape)

(225, 5)
(225, 5)


In [27]:
df_stations_explored = df_stations.copy()
df_stations_explored['Comp'] = np.nan
df_stations_explored['Cust'] = np.nan

i = 0
for row in df_stations.iterrows():
    #Debug
    i = i+1
    if i==250:
        break
 
    print('{} - Serach {}'.format(i, row[1]['Name']))
    df_FS_cust = get_surrounding_from_fs(row, catId_customer, 300)
    df_FS_comp = get_surrounding_from_fs(row, catId_competitor,150)
    
    df_stations_explored.loc[row[0], 'Cust'] = df_FS_cust.shape[0]
    df_stations_explored.loc[row[0], 'Comp'] = df_FS_comp.shape[0]
    
    if df_FS_cust.empty == False:
        for lat, long, name in zip(df_FS_cust['lat'], df_FS_cust['lng'], df_FS_cust['name']):
            label = '{}'.format(name)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [float(lat), float(long)],
                radius=3,
                popup=label,
                color='green',
                fill=True,
                fill_color='green',
                fill_opacity=1.0,
                parse_html=False).add_to(k_map)  
    
    if df_FS_comp.empty == False:
        for lat, long, name in zip(df_FS_comp['lat'], df_FS_comp['lng'], df_FS_comp['name']):
            label = '{}'.format(name)
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [float(lat), float(long)],
                radius=3,
                popup=label,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=1.0,
                parse_html=False).add_to(k_map) 
        
k_map

1 - Serach Heumarkt
2 - Serach Neumarkt
3 - Serach Poststr.
4 - Serach Mauritiuskirche
5 - Serach Rathaus
6 - Serach Appellhofplatz
7 - Serach Dom/Hbf
8 - Serach Breslauer Platz/Hbf
9 - Serach Severinstr.
10 - Serach Ubierring
11 - Serach Chlodwigplatz
12 - Serach Ulrepforte
13 - Serach Bonner Wall
14 - Serach Eifelplatz
15 - Serach Eifelstr.
16 - Serach Barbarossaplatz
17 - Serach Zülpicher Platz
18 - Serach Dasselstr./Bf Süd
19 - Serach Eifelwall
20 - Serach Rudolfplatz
21 - Serach Moltkestr.
22 - Serach Friesenplatz
23 - Serach Hans-Böckler-Platz/Bf West
24 - Serach Christophstr./Mediapark
25 - Serach Reichenspergerplatz
26 - Serach Ebertplatz
27 - Serach Hansaring
28 - Serach Deutzer Freiheit
29 - Serach Suevenstr.
30 - Serach Bf Deutz/Messe
31 - Serach Koelnmesse
32 - Serach Deutz Technische Hochschule
33 - Serach Severinsbrücke
34 - Serach Drehbrücke
35 - Serach Poller Kirchweg
36 - Serach Bf Deutz/LANXESS arena
37 - Serach Pohligstr.
38 - Serach Herthastr.
39 - Serach Gottesweg


The information density makes it imposible to read anything usefull on preferable locations for our business from the map. So we use an easy metric to calculate a suitability index for a potential cafes of ours for each stations.<br>
<br>
_Metric: (Number of tram lines stopping at the station + numer of high customer sites) / number of competitors_<br>
<br>
This value we will then use for the radius of each circle for a station on the map. The bigger the circle the better the location.

In [28]:
# Apply metric to calculate a suitability index
met = []

for row in df_stations_explored.iterrows():
    met.append((row[1]['Lines count'] + row[1]['Cust'])/(row[1]['Comp']+1))
    
df_stations_explored['Suitability'] = met
df_stations_explored.sort_values(by=['Suitability'], ascending=False, inplace=True)
df_stations_explored.head()

,Name,Long,Lat,Lines count,Distance,Comp,Cust,Suitability
54,Universität,6.9312003766,50.9259899857,1,2.475556,0.0,22.0,23.0
11,Ulrepforte,6.951437134,50.9243235691,2,1.677367,0.0,8.0,10.0
31,Deutz Technische Hochschule,6.9859264567,50.9365857842,2,1.866090,0.0,6.0,8.0
2,Poststr.,6.9500805214,50.9316916582,4,1.025377,0.0,2.0,6.0
18,Eifelwall,6.937176778,50.924873049,1,2.215972,1.0,9.0,5.0


# Results

Now lets have a look at the map with the suitability index applied. Additionally we colorize the stations according to their suitability index. We assuem that the company is willing to open up to 20 cafes in total at the best locations determined. We use a traffic light labeling to highlight the stations to choose:
- green for the best 10 stations
- orange for the second best 10 stations
- red for the rest which are uninteressted to the company

For better visibility of the single markers on the map, we set the map apperance to a darker shade.

In [29]:
k_map2 = folium.Map(location=[k_lat, k_long], zoom_start=13)

i = 0
for lat, long, name, comp, cust, lines, suitIn in zip(df_stations_explored['Lat'], df_stations_explored['Long'], df_stations_explored['Name'], df_stations_explored['Comp'], df_stations_explored['Cust'], df_stations_explored['Lines count'], df_stations_explored['Suitability']):
    
    if i <10:
        c = 'limegreen'
    elif i < 20:
        c = 'orange'
    else:
        c = 'red'
    
    i = i+1
    
    label = '{}: Comp: {} - Cust: {} - Lines: {} - Suitability: {}'.format(name, comp, cust, lines, suitIn)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(long)],
        radius=suitIn,
        popup=label,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=1.0,
        parse_html=False).add_to(k_map2)  

folium.TileLayer('cartodbdark_matter').add_to(k_map2)
    
k_map2

The best 10 locations for the cafés are

In [30]:
df_stations_explored.iloc[0:10]

,Name,Long,Lat,Lines count,Distance,Comp,Cust,Suitability
54,Universität,6.9312003766,50.9259899857,1,2.475556,0.0,22.0,23.0
11,Ulrepforte,6.951437134,50.9243235691,2,1.677367,0.0,8.0,10.0
31,Deutz Technische Hochschule,6.9859264567,50.9365857842,2,1.866090,0.0,6.0,8.0
2,Poststr.,6.9500805214,50.9316916582,4,1.025377,0.0,2.0,6.0
18,Eifelwall,6.937176778,50.924873049,1,2.215972,1.0,9.0,5.0
49,Universitätsstr.,6.9242751929,50.936653862,2,2.559527,0.0,3.0,5.0
52,Arnulfstr.,6.9305212496,50.918530254,1,3.050616,0.0,4.0,5.0
51,Weißhausstr.,6.9337951463,50.921716127,1,2.633557,1.0,6.0,3.5
104,Lohsestr.,6.9551338824,50.9585057929,2,2.268666,0.0,1.0,3.0
25,Ebertplatz,6.9595370283,50.9510131543,2,1.408531,0.0,1.0,3.0


The second 10 best locations for the cafés are

In [31]:
df_stations_explored.iloc[10:20]

,Name,Long,Lat,Lines count,Distance,Comp,Cust,Suitability
145,Ostheim,7.042401237,50.9291592467,1,5.981872,0.0,2.0,3.0
160,Bf Mülheim,7.0128560514,50.9569283058,2,4.308951,0.0,1.0,3.0
44,Siegstr.,6.99488326,50.8830097,2,6.647034,0.0,1.0,3.0
156,Stegerwaldsiedlung,6.9933994166,50.9485947332,2,2.647463,0.0,1.0,3.0
72,Oskar-Jäger-Str./Gürtel,6.9105605258,50.9393562796,1,3.534800,0.0,2.0,3.0
208,Brühl Nord,6.9018038124,50.8351066781,1,12.221681,0.0,2.0,3.0
41,Schönhauser Str.,6.9717962172,50.9145492339,2,2.781786,0.0,1.0,3.0
50,Melaten,6.917048536,50.936734511,2,3.074505,0.0,1.0,3.0
36,Pohligstr.,6.9415800881,50.9166833148,1,2.747891,0.0,2.0,3.0
102,Neusser Str./Gürtel,6.9513443152,50.972558269,2,3.855845,1.0,4.0,3.0


# Discussion

# Conclusion